This notebook looks for files like `task-5e53f31f4e39270010025f6f.json` from Scale, then downloads, renames, and formats the labels consistent with the existing dataset in Pascal VOC format.

In [1]:
import os
import cv2
import json
import glob
import urllib.request

In [2]:
import os.path as osp

import numpy as np
import PIL.Image


def lblsave(filename, lbl):
    import imgviz

    if osp.splitext(filename)[1] != '.png':
        filename += '.png'
    # Assume label ranges [-1, 254] for int32,
    # and [0, 255] for uint8 as VOC.
    if lbl.min() >= -1 and lbl.max() < 255:
        lbl_pil = PIL.Image.fromarray(lbl.astype(np.uint8), mode='P')
        colormap = imgviz.label_colormap()
        lbl_pil.putpalette(colormap.flatten())
        lbl_pil.save(filename)
    else:
        raise ValueError(
            '[%s] Cannot save the pixel-wise class label as PNG. '
            'Please consider using the .npy format.' % filename
        )

In [3]:
#scale_tasks_path = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/GLNI/land/scale/' # 18
#scale_tasks_path = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/GLNI/port/scale/' # 4
#scale_tasks_path = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/WHERD/scale/' # 6
#scale_tasks_path = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/Tripod/503/' # 1
#scale_tasks_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Train/2017-08-bak-16-good-labels/' # 3
scale_tasks_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Lab' # 3
task_list = glob.glob(osp.join(scale_tasks_path, 'task-*.json'))
print(len(task_list))
task_list

1


['/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Lab/task-5e53f31d599b6700162ce302.json']

In [4]:
for i in range(len(task_list)):

    with open(task_list[i], 'r') as f:
        att = json.load(f)

    source_image_name = att['params']['attachment']
    source_image_root = source_image_name.split('/')[-1].split('.')[0]

    outfile = os.path.join(scale_tasks_path, source_image_root)

    """This will download the pink and green mask from scale, 
    but this is overwritten by the VOC format mask."""
    file, http_msg = urllib.request.urlretrieve(
        att['response']['annotations']['combined']['image'], filename=outfile + '_scale.png')

    np_img = cv2.imread(outfile + '_scale.png') # in BlueGreenRed (BGR) format

    lbl = np.zeros((np_img.shape[0], np_img.shape[1]))
    lbl[(np_img[:, :, 2] ==  60)] = 1

    lblsave(outfile + '.png', lbl) # save as indexed color RGB image
    #cv2.imwrite(outfile + '.png', np_img) # this is incorrect and saves as a normal RGB image
    print('Saved item %d' % i, outfile)

Saved item 0 /scratch/ssd/gallowaa/cciw/dataset_raw/Test/Lab/Lab_3539-1_2018-08-07_image-1


# End notebook - begin scratchspace

In [ ]:
#att['response']['annotations']['combined']['image']

In [ ]:
source_image_name = att['params']['attachment']
source_image_root = source_image_name.split('/')[-1].split('.')[0]
print(source_image_root)

In [ ]:
outfile = os.path.join(scale_tasks_path, source_image_root + '.png')

file, http_msg = urllib.request.urlretrieve(
    att['response']['annotations']['combined']['image'], filename=outfile)
print(file)
print(http_msg)

In [ ]:
import PIL.Image
#import numpy as np

In [ ]:
#lab = PIL.Image.open(outfile)

In [ ]:
path = '/scratch/ssd/gallowaa/cciw/VOCdevkit/VOC2012_v023/'
label_path = os.path.join(path, 'SegmentationClassPNG')
list_of_labels = os.listdir(label_path)
list_of_labels[0]
voc_mask = os.path.join(label_path, list_of_labels[0])

In [ ]:
scale_img = cv2.imread('/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/Tripod/1354/Lab_1354_2019-05-24_GoPro-1.png')

In [ ]:
plt.imshow(scale_img)

In [ ]:
plt.imshow(voc_img)

In [ ]:
voc_img = cv2.imread(voc_mask)

In [ ]:
voc_img.shape

In [ ]:
np.unique(scale_img) # (75, 25, 230) - bkg
                  # (75, 180, 60) - mussel

In [ ]:
np_img = cv2.imread(outfile)

In [ ]:
out_img = np_img.copy()

In [ ]:
out_img[(np_img[:, :, 2] == 60)] = (0, 0, 128)

In [ ]:
out_img[(np_img[:, :, 2] == 230)] = (0, 0, 0)

In [ ]:
plt.imshow(out_img)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
cv2.imwrite(outfile, out_img)

In [ ]:
outfile

In [ ]:
import numpy as np